In [2]:
import os
import sys
import datetime

import matlab
import pandas as pd
import numpy as np
import csv

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/";

sys.path.append("../matlab/myBTD2/myBTD2/for_redistribution_files_only");
import myBTD2

sys.path.append("../matlab/myBTD/myBTD/for_redistribution_files_only");
import myBTD

from myPackages import myDehankelization, myHankelization

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
aioros_df = pd.read_pickle(f'../workdata/aioros/aioros.pkl')
aioros_df.shape

(57, 4)

In [15]:
def get_dim_hankel(N):
    N = int(N)
    if N & 1:
        I = J = (N + 1)/2 
    else:
        I = N/2
        J = N/2 + 1
    return int(I), int(J)


def hankelization(recording):
    n_lead, n_rec = recording.shape
    I, J = get_dim_hankel(n_rec)
    T = np.zeros((I, J, n_lead))
    my_hankelization = myHankelization.initialize()
    for i in range(0, n_lead):
        vec = matlab.double(recording[i].tolist(), size=recording[i].shape)
        T[:, :, i] = my_hankelization.hankelization(vec)
    my_hankelization.terminate()
    T = matlab.double(T.tolist(), size=T.shape)
    return T


def run_btd(params):
    btd = myBTD.initialize()
    result, output = btd.myBTD(params['rank'], 
                               params['multirank'], 
                               params['max_iter'], 
                               params['tol_fun'], 
                               params['tol_x'], 
                               params['display'], 
                               params['recording_hankel'],
                               nargout=2)
    btd.terminate()
    return result, output


def run_btd2(params):
    btd = myBTD2.initialize()
    result, output = btd.myBTD2(params['rank'], 
                                params['lr'], 
                                params['max_iter'], 
                                params['tol_fun'], 
                                params['tol_x'], 
                                params['display'], 
                                params['recording_hankel'], 
                                nargout=2)
    btd.terminate()
    return result, output

In [20]:
MULTIRANK = matlab.double([40, 40, 1], size=(1, 3))
LR = matlab.double([17], size=(1, 1))
MAX_ITER = matlab.double([1000], size=(1, 1))
TOLERANCE_FUN = matlab.double([1.0E-6], size=(1, 1))
TOLERANCE_X = matlab.double([1.0E-6], size=(1, 1))
DISPLAY = matlab.logical([False], size=(1, 1))
path = f'../workdata/shura/'

LL1_MODE = True

if not os.path.exists(f'{path}'):
    os.makedirs(path)  

aioros_df = pd.read_pickle(f'../workdata/aioros/aioros_large.pkl')

In [6]:
aioros_df = pd.read_pickle(f'../workdata/aioros/aioros_large.pkl')
aioros_df.groupby(['diag', 'db']).size()

diag  db                  
AF    WFDB_CPSC2018            356
      WFDB_CPSC2018_2           47
      WFDB_ChapmanShaoxing     406
      WFDB_Ga                  179
      WFDB_PTB                   4
      WFDB_PTBXL               440
      WFDB_StPetersburg          1
SR    WFDB_CPSC2018            291
      WFDB_ChapmanShaoxing     428
      WFDB_Ga                  535
      WFDB_Ningbo             1792
      WFDB_PTB                  25
      WFDB_PTBXL              5390
dtype: int64

In [ ]:
rows = aioros_df.shape[0]

for rank in range(6, 7):
    print(f'RANK: {rank}')
    
    if not os.path.exists(f'{path}/rank_{rank}'):
        os.makedirs(f'{path}/rank_{rank}')  
        
    rank_matlab = matlab.double([rank], size=(1, 1))
    
    for index, row in aioros_df.iterrows():
        recording = row['data']
        recording_hankel = hankelization(recording)
        print(f'{index}/{rows}')
                
        params = {
            'rank': rank_matlab,
            'lr': LR,
            'multirank': MULTIRANK,
            'max_iter': MAX_ITER,
            'tol_fun': TOLERANCE_FUN,
            'tol_x': TOLERANCE_X,
            'display': DISPLAY,
            'recording_hankel': recording_hankel
        }
        
        result, output = run_btd2(params) if LL1_MODE else run_btd(params)
        
        output['index'] = index
        output['rank'] = rank
        output['lr'] = LR[0][0]
        output['filename'] = row['filename']                       
        output['db'] = row['db']
        output['diagnostic'] = row['diag']
        output['datetime'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        with open(f'{path}/rank_{rank}/{row.name}.npy', 'wb') as file:
            np.save(file, result)

        file_exists = os.path.exists(f'{path}/results.csv')
        
        with open(f'{path}/results.csv', 'a') as f:
            headers = output.keys()
            writer = csv.DictWriter(f, delimiter=',', lineterminator='\n', fieldnames=headers)
            if not file_exists:
                writer.writeheader()
            writer.writerow(output)

RANK: 6
0/9894


/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


1/9894
2/9894
3/9894
4/9894
5/9894
6/9894
7/9894
8/9894
9/9894
10/9894
11/9894
12/9894
13/9894
14/9894
15/9894
16/9894
17/9894
18/9894
19/9894
20/9894
21/9894
22/9894
23/9894
24/9894
25/9894
26/9894
27/9894
28/9894
29/9894
30/9894
31/9894
32/9894
33/9894
34/9894
35/9894
36/9894
37/9894
38/9894
39/9894
40/9894
41/9894
42/9894
43/9894
44/9894
45/9894
46/9894
47/9894
48/9894
49/9894
50/9894
51/9894
52/9894
53/9894
54/9894
55/9894
56/9894
57/9894
58/9894
59/9894
60/9894
61/9894
62/9894
63/9894
64/9894
65/9894
66/9894
67/9894
68/9894
69/9894
70/9894
71/9894
72/9894
73/9894
74/9894
75/9894
76/9894
77/9894
78/9894
79/9894
80/9894
81/9894
82/9894
83/9894
84/9894
85/9894
86/9894
87/9894
88/9894
89/9894
90/9894
91/9894
92/9894
93/9894
94/9894
95/9894
96/9894
97/9894
98/9894
99/9894
100/9894
101/9894
102/9894
103/9894
104/9894
105/9894
106/9894
107/9894
108/9894
109/9894
110/9894
111/9894
112/9894
113/9894
114/9894
115/9894
116/9894
117/9894
118/9894
119/9894
120/9894
121/9894
122/9894
123/9894
1

924/9894
925/9894
926/9894
927/9894
928/9894
929/9894
930/9894
931/9894
932/9894
933/9894
934/9894
935/9894
936/9894
937/9894
938/9894
939/9894
940/9894
941/9894
942/9894
943/9894
944/9894
945/9894
946/9894
947/9894
948/9894
949/9894
950/9894
951/9894
952/9894
953/9894
954/9894
955/9894
956/9894
957/9894
958/9894
959/9894
960/9894
961/9894
962/9894
963/9894
964/9894
965/9894
966/9894
967/9894
968/9894
969/9894
970/9894
971/9894
972/9894
973/9894
974/9894
975/9894
976/9894
977/9894
978/9894
979/9894
980/9894
981/9894
982/9894
983/9894
984/9894
985/9894
986/9894
987/9894
988/9894
989/9894
990/9894
991/9894
992/9894
993/9894
994/9894
995/9894
996/9894
997/9894
998/9894
999/9894
1000/9894
1001/9894
1002/9894
1003/9894
1004/9894
1005/9894
1006/9894
1007/9894
1008/9894
1009/9894
1010/9894
1011/9894
1012/9894
1013/9894
1014/9894
1015/9894
1016/9894
1017/9894
1018/9894
1019/9894
1020/9894
1021/9894
1022/9894
1023/9894
1024/9894
1025/9894
1026/9894
1027/9894
1028/9894
1029/9894
1030/9894
1031/9

1751/9894
1752/9894
1753/9894
1754/9894
1755/9894
1756/9894
1757/9894
1758/9894
1759/9894
1760/9894
1761/9894
1762/9894
1763/9894
1764/9894
1765/9894
1766/9894
1767/9894
1768/9894
1769/9894
1770/9894
1771/9894
1772/9894
1773/9894
1774/9894
1775/9894
1776/9894
1777/9894
1778/9894
1779/9894
1780/9894
1781/9894
1782/9894
1783/9894
1784/9894
1785/9894
1786/9894
1787/9894
1788/9894
1789/9894
1790/9894
1791/9894
1792/9894
1793/9894
1794/9894
1795/9894
1796/9894
1797/9894
1798/9894
1799/9894
1800/9894
1801/9894
1802/9894
1803/9894
1804/9894
1805/9894
1806/9894
1807/9894
1808/9894
1809/9894
1810/9894
1811/9894
1812/9894
1813/9894
1814/9894
1815/9894
1816/9894
1817/9894
1818/9894
1819/9894
1820/9894
1821/9894
1822/9894
1823/9894
1824/9894
1825/9894
1826/9894
1827/9894
1828/9894
1829/9894
1830/9894
1831/9894
1832/9894
1833/9894
1834/9894
1835/9894
1836/9894
1837/9894
1838/9894
1839/9894
1840/9894
1841/9894
1842/9894
1843/9894
1844/9894
1845/9894
1846/9894
1847/9894
1848/9894
1849/9894
1850/9894


...

4211/9894
4212/9894
4213/9894
4214/9894
4215/9894
4216/9894
4217/9894
4218/9894
4219/9894
4220/9894
4221/9894
4222/9894
4223/9894
4224/9894
4225/9894
4226/9894
4227/9894
4228/9894
4229/9894
4230/9894
4231/9894
4232/9894
4233/9894
4234/9894
4235/9894
4236/9894
4237/9894
4238/9894
4239/9894
4240/9894
4241/9894
4242/9894
4243/9894
4244/9894
4245/9894
4246/9894
4247/9894
4248/9894
4249/9894
4250/9894
4251/9894
4252/9894
4253/9894
4254/9894
4255/9894
4256/9894
4257/9894
4258/9894
4259/9894
4260/9894
4261/9894
4262/9894
4263/9894
4264/9894
4265/9894
4266/9894
4267/9894
4268/9894
4269/9894
4270/9894
4271/9894
4272/9894
4273/9894
4274/9894
4275/9894
4276/9894
4277/9894
4278/9894
4279/9894
4280/9894
4281/9894
4282/9894
4283/9894
4284/9894
4285/9894
4286/9894
4287/9894
4288/9894
4289/9894
4290/9894
4291/9894
4292/9894
4293/9894
4294/9894
4295/9894
4296/9894
4297/9894
4298/9894
4299/9894
4300/9894
4301/9894
4302/9894
4303/9894
4304/9894
4305/9894
4306/9894
4307/9894
4308/9894
4309/9894
4310/9894


5031/9894
5032/9894
5033/9894
5034/9894
5035/9894
5036/9894
5037/9894
5038/9894
5039/9894
5040/9894
5041/9894
5042/9894
5043/9894
5044/9894
5045/9894
5046/9894
5047/9894
5048/9894
5049/9894
5050/9894
5051/9894
5052/9894
5053/9894
5054/9894
5055/9894
5056/9894
5057/9894
5058/9894
5059/9894
5060/9894
5061/9894
5062/9894
5063/9894
5064/9894
5065/9894
5066/9894
5067/9894
5068/9894
5069/9894
5070/9894
5071/9894
5072/9894
5073/9894
5074/9894
5075/9894
5076/9894
5077/9894
5078/9894
5079/9894
5080/9894
5081/9894
5082/9894
5083/9894
5084/9894
5085/9894
5086/9894
5087/9894
5088/9894
5089/9894
5090/9894
5091/9894
5092/9894
5093/9894
5094/9894
5095/9894
5096/9894
5097/9894
5098/9894
5099/9894
5100/9894
5101/9894
5102/9894
5103/9894
5104/9894
5105/9894
5106/9894
5107/9894
5108/9894
5109/9894
5110/9894
5111/9894
5112/9894
5113/9894
5114/9894
5115/9894
5116/9894
5117/9894
5118/9894
5119/9894
5120/9894
5121/9894
5122/9894
5123/9894
5124/9894
5125/9894
5126/9894
5127/9894
5128/9894
5129/9894
5130/9894


5851/9894
5852/9894
5853/9894
5854/9894
5855/9894
5856/9894
5857/9894
5858/9894
5859/9894
5860/9894
5861/9894
5862/9894
5863/9894
5864/9894
5865/9894
5866/9894
5867/9894
5868/9894
5869/9894
5870/9894
5871/9894
5872/9894
5873/9894
5874/9894
5875/9894
5876/9894
5877/9894
5878/9894
5879/9894
5880/9894
5881/9894
5882/9894
5883/9894
5884/9894
5885/9894
5886/9894
5887/9894
5888/9894
5889/9894
5890/9894
5891/9894
5892/9894
5893/9894
5894/9894
5895/9894
5896/9894
5897/9894
5898/9894
5899/9894
5900/9894
5901/9894
5902/9894
5903/9894
5904/9894
5905/9894
5906/9894
5907/9894
5908/9894
5909/9894
5910/9894
5911/9894
5912/9894
5913/9894
5914/9894
5915/9894
5916/9894
5917/9894
5918/9894
5919/9894
5920/9894
5921/9894
5922/9894
5923/9894
5924/9894
5925/9894
5926/9894
5927/9894
5928/9894
5929/9894
5930/9894
5931/9894
5932/9894
5933/9894
5934/9894
5935/9894
5936/9894
5937/9894
5938/9894
5939/9894
5940/9894
5941/9894
5942/9894
5943/9894
5944/9894
5945/9894
5946/9894
5947/9894
5948/9894
5949/9894
5950/9894


6671/9894
6672/9894
6673/9894
6674/9894
6675/9894
6676/9894
6677/9894
6678/9894
6679/9894
6680/9894
6681/9894
6682/9894
6683/9894
6684/9894
6685/9894
6686/9894
6687/9894
6688/9894
6689/9894
6690/9894
6691/9894
6692/9894
6693/9894
6694/9894
6695/9894
6696/9894
6697/9894
6698/9894
6699/9894
6700/9894
6701/9894
6702/9894
6703/9894
6704/9894
6705/9894
6706/9894
6707/9894
6708/9894
6709/9894
6710/9894
6711/9894
6712/9894
6713/9894
6714/9894
6715/9894
6716/9894
6717/9894
6718/9894
6719/9894
6720/9894
6721/9894
6722/9894
6723/9894
6724/9894
6725/9894
6726/9894
6727/9894
6728/9894
6729/9894
6730/9894
6731/9894
6732/9894
6733/9894
6734/9894
6735/9894
6736/9894
6737/9894
6738/9894
6739/9894
6740/9894
6741/9894
6742/9894
6743/9894
6744/9894
6745/9894
6746/9894
6747/9894
6748/9894
6749/9894
6750/9894
6751/9894
6752/9894
6753/9894
6754/9894
6755/9894
6756/9894
6757/9894
6758/9894
6759/9894
6760/9894
6761/9894
6762/9894
6763/9894
6764/9894
6765/9894
6766/9894
6767/9894
6768/9894
6769/9894
6770/9894


7491/9894
7492/9894
7493/9894
7494/9894
7495/9894
7496/9894
7497/9894
7498/9894
7499/9894
7500/9894
7501/9894
7502/9894
7503/9894
7504/9894
7505/9894
7506/9894
7507/9894
7508/9894
7509/9894
7510/9894
7511/9894
7512/9894
7513/9894
7514/9894
7515/9894
7516/9894
7517/9894
7518/9894
7519/9894
7520/9894
7521/9894
7522/9894
7523/9894
7524/9894
7525/9894
7526/9894
7527/9894
7528/9894
7529/9894
7530/9894
7531/9894
7532/9894
7533/9894
7534/9894
7535/9894
7536/9894
7537/9894
7538/9894
7539/9894
7540/9894
7541/9894
7542/9894
7543/9894
7544/9894
7545/9894
7546/9894
7547/9894
7548/9894
7549/9894
7550/9894
7551/9894
7552/9894
7553/9894
7554/9894
7555/9894
7556/9894
7557/9894
7558/9894
7559/9894
7560/9894
7561/9894
7562/9894
7563/9894
7564/9894
7565/9894
7566/9894
7567/9894
7568/9894
7569/9894
7570/9894
7571/9894
7572/9894
7573/9894
7574/9894
7575/9894
7576/9894
7577/9894
7578/9894
7579/9894
7580/9894
7581/9894
7582/9894
7583/9894
7584/9894
7585/9894
7586/9894
7587/9894
7588/9894
7589/9894
7590/9894
